In [1]:
#Auxiliary notebook to run from A3_jonathan
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
n_steps_factor = 1

The file_gams_py_gdb0.gdx is still active and was not deleted.
The file_gams_py_gdb1.gdx is still active and was not deleted.
The file_gams_py_gdb2.gdx is still active and was not deleted.
The file_gams_py_gdb3.gdx is still active and was not deleted.
The file_gams_py_gdb5.gdx is still active and was not deleted.
The file_gams_py_gdb7.gdx is still active and was not deleted.


In [2]:
if False:
    catalog = "categoryexperiment2_ID"
    currapp = False
    elec = False
    smoothed = False
    os.chdir(directory['py'])
    import techdata_to_tree, sys, ShockFunction
    import abatement_EOP_new as abatement
    import ShockFunction
    from DB2Gams import OrdSet as OS
    os.chdir(directory['curr'])
    with open(os.getcwd()+'\\conopt_ExtraPivot.opt',"r") as file:
        kwargs_mi = {'opt_file': file.read()}

In [3]:
def organize_sol_data(sols, x_var, nodes_to_save, node_to_shock, tech_catalog):
#     warnings.filterwarnings("error", UserWarning)
    list_of_dfs = []
    for var in sols:
        if var == x_var:
            continue
        df = sols[var]
        index_name = df.index.names[1]
        if len(df.index.get_level_values(index_name).unique()) == 1:
            only_one_series = True
        else:
            only_one_series = False
        df = df.reset_index()
        df2 = df.pivot_table(df.columns[-1], df.columns[0], df.columns[1]).reset_index()
        if tech_catalog in nodes_to_save and not only_one_series:
            df2 = df2[[x_var + "_loop"] + [c for c in nodes_to_save[tech_catalog] if c in df2.columns]]
        df3 = df2.merge(sols[x_var].iloc[sols[x_var].index.get_level_values(1) == node_to_shock].reset_index().drop(columns=sols[x_var].index.names[1]), \
                        how="inner", on=x_var+"_loop").drop(columns=x_var+"_loop")
        df3 = df3.rename(columns={"sol_"+x_var:x_var}).set_index(x_var).sort_values(by=x_var)
        if only_one_series:
            df3.rename(columns={df3.columns[0]:var}, inplace=True)
        else:
            df3.rename(columns={key:var + "_"+ key for key in df3.columns}, inplace=True)
        list_of_dfs.append(df3)
    return list_of_dfs

In [4]:
out = {}

#### Load and setup

In [5]:
#Load model
if elec:
    elec_string = "_elec"
else:
    elec_string = ""
if smoothed:
    smoothed_string = "_smoothed"
else:
    smoothed_string = ""
gm = abatement.abate(pickle_path=directory["gams"] + "/" + catalog + elec_string + smoothed_string, work_folder=directory['work'])
gm.reset_settings()
gm.add_functions()
if catalog in ["main_catalog", "categoryexperiment_EOP", "categoryexperiment2_EOP", "categoryexperiment2_ID"]: #categoryexperiment2 always has an EOP tech
    if elec:
        gm.setstate("EOP_elec")
    else:
        gm.setstate("EOP")
elif catalog in ["categoryexperiment_ID"]:
    gm.setstate("ID")

In [6]:
name = "CO2tax"
x_var = "pM"
node_to_shock = "CO2"

In [7]:
vars_ = ["M0", "PbT", "pM", "PwThat", "qD", "PwT", "qS"]
if "EOP" in gm.state:
    vars_ += ["M"]
if currapp:
    vars_ += ["muG", "sigmaG"]
store_sol = {var:{"domains":x_var + '_loop'} for var in vars_}

In [8]:
db_end_of_loop = DataBase.GPM_database()
db_end_of_loop[x_var] = gm.get(x_var).copy()
if "categoryexperiment2" in catalog:
    db_end_of_loop[x_var].vals[node_to_shock] = db_end_of_loop[x_var].vals[node_to_shock]*0 + 10
else:
    db_end_of_loop[x_var].vals[node_to_shock] = db_end_of_loop[x_var].vals[node_to_shock]*0 + 20

In [9]:
gm.write_and_run(name=name,kwargs_init={'check_variables':True},add_checkpoint=name, kwargs_mi=kwargs_mi)
gm.checkpoints[name + "_cp"] = gm.model_instances[name].ws.add_checkpoint()

In [10]:
print(gm.model_instances[name].solve_sneakily(db_star=db_end_of_loop,from_cp = True, cp_init = gm.checkpoints[name], 
                                              options_run={'checkpoint': gm.checkpoints[name + '_cp']}, 
                                              kwargs_shock={"n_steps":int(500*n_steps_factor), "loop_name":x_var + "_loop", "store_sol":store_sol}))

{'Modelstat': 16.0, 'Solvestat': 1.0}


In [11]:
mi = gm.model_instances[name]
assert mi.modelstat

In [12]:
sols = {key:mi.out_db.series["sol_" + key].vals for key in store_sol.keys()}
nodes_to_save = {
    "main_catalog":["U_ID_t2_1", "U_EOP_t1_1", "U_EOP_t2_1", "U_ID_t1_2", "C_EH_2", "U0_ID_C_EH_2", "CO2", "basetech_EH", "basetech_EH_oil", 
                    "oil", "U_ID_t1_1", "U_ID_t2_2", "U0_ID_C_EH_1", "U0_ID_C_EH_3", "ID_t1", "ID_t2", "C_CO2_1", "C_CO2_2", "basetech_EH_electricity"],
    "categoryexperiment_EOP":["U_EOP_t1_1", "CO2", "oil", "basetech_EH_oil"],
    "categoryexperiment_ID":["U_ID_t1_1", "CO2", "oil", "basetech_EH_oil", "U0_ID_C_EH_2", "C_EH_2", "basetech_EH"],
    "categoryexperiment2_EOP":["U_EOP_t1_1", "U_EOP_t2_1", "CO2", "oil", "basetech_EH_oil"],
    "categoryexperiment2_ID":["U_EOP_t1_1", "U_ID_t2_1", "CO2", "oil", "basetech_EH_oil", "U0_ID_C_EH_2", "C_EH_2", "basetech_EH"]
}

In [13]:
gm.model.database.merge_dbs(gm.model.database,mi.out_db,'second')

In [14]:
out[name] = organize_sol_data(sols, x_var, nodes_to_save, node_to_shock, gm.model.settings.catalog)

In [15]:
df_out = pd.concat(out[name], axis = 1)
df_out.to_pickle(x_var + "_" + gm.model.settings.catalog + elec_string + smoothed_string + ".pkl")

In [17]:
df_out

,M0,PwThat_U_EOP_t1_1,PwThat_U_ID_t2_1,PwThat_basetech_EH_oil,PwThat_U0_ID_C_EH_2,PwThat_C_EH_2,PwThat_basetech_EH,qD_U_EOP_t1_1,qD_U_ID_t2_1,qD_oil,qD_basetech_EH_oil,qD_U0_ID_C_EH_2,qD_C_EH_2,qD_basetech_EH,PwT_basetech_EH_oil,M
pM,,,,,,,,,,,,,,,,
0.500000,36.040000,3.50054,7.00594,1.155404,3.711283,3.711283,3.711283,0.000001,0.000001,17.000000,16.500000,25.000000,25.000000,50.000002,0.1,36.040000
0.696970,32.281455,3.50054,7.00594,1.569435,3.840358,3.840358,3.840358,0.000001,0.000001,15.227101,14.797363,24.997995,24.997995,49.995991,0.1,32.281455
0.893939,29.733305,3.50054,7.00594,1.981282,3.957235,3.957235,3.957235,0.000001,0.000001,14.025144,13.642103,24.996272,24.996272,49.992547,0.1,29.733305
1.090909,27.859980,3.50054,7.00594,2.389997,4.065141,4.065141,4.065141,0.000002,0.000001,13.141500,12.792283,24.994756,24.994756,49.989514,0.1,27.859980
1.287879,26.410313,3.50054,7.00594,2.794468,4.165887,4.165887,4.165887,0.000007,0.000001,12.457695,12.134349,24.993400,24.993400,49.986802,0.1,26.410307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19.212121,7.101452,3.50054,7.00594,24.508507,7.617434,7.005940,7.617434,3.479713,24.969238,3.349742,3.237156,0.000001,24.969238,24.967152,0.1,3.621741
19.409091,7.086623,3.50054,7.00594,24.721470,7.645018,7.005940,7.645018,3.472446,24.969221,3.342747,3.230626,0.000001,24.969222,24.967045,0.1,3.614178
19.606061,7.071982,3.50054,7.00594,24.934434,7.672547,7.005940,7.672547,3.465272,24.969205,3.335841,3.224180,0.000001,24.969205,24.966939,0.1,3.606711
